In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
target_url = 'https://webapps2.rrc.texas.gov/EWA/drillingPermitsQueryAction.do'

In [3]:
api_list = [4212335341,4238940705,4238940783] #No Ammendments, Ammnedment filed no approval date, one ammendment

#### First Try

In [4]:
def payload(api):
         
    """      
    Parameters
    ----------
    api : TYPE, int
        DESCRIPTION. Strips API of initial '42'.

    Returns
    -------
    Dict.
    """
    api = int(str(api)[2:])

    payload = {
    'methodToCall': 'search',
    'searchArgs.apiNoHndlr.inputValue': api,
    'searchArgs.operatorNameWildcardHndlr.inputValue': 'beginsWith',
    'searchArgs.leaseNameWildcardHndlr.inputValue': 'beginsWith',
    'searchArgs.fieldNameWildcardHndlr.inputValue': 'beginsWith',
    'searchArgs.surveyNameWildcardHndlr.inputValue': 'beginsWith'
    }

    return payload

In [5]:
def parse_data(post_response):
    """      
    Parameters
    ----------
    post_response : TYPE, Requests Response
        DESCRIPTION. Response from the target url.

    Returns
    -------
    DataFrame.
    """
    soup = BeautifulSoup(post_response.content,'html.parser')
    web_links = soup.select('a')
    href = [web_link['href'] for web_link in web_links if 'drillingPermitDetailAction' in web_link['href']]

    df = pd.read_html(post_response.text)[9]
    df.columns = df.iloc[1,:]
    df = df.drop([0,1]).copy()
    df = df.reset_index(drop=True).copy()
    df['API NO.'] = int(df['API NO.'].str.split()[0][0])
    df['HREF'] = href

    return df

In [6]:
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'Host': 'webapps2.rrc.texas.gov',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
    'sec-ch-ua': "Google Chrome;v='119', 'Chromium';v='119', 'Not?A_Brand';v='24'",
    'sec-ch-ua-mobile': "?0",
    'sec-ch-ua-platform': "Windows"
}

In [13]:
try:
    with requests.session() as s:
        res_get = s.get(url=target_url,headers=headers,verify=False)

        if res_get.status_code == 200:
            cookies = res_get.cookies
            res_post = s.post(url=target_url,data=payload(api_list[1]),cookies=cookies,headers=headers)

except Exception as e:
    print(e)

c:\Users\Apoorva.Saxena\OneDrive - Sitio Royalties\Desktop\Project - Apoorva\Python\Scraping\RRC\scraper\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'webapps2.rrc.texas.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Apoorva.Saxena\OneDrive - Sitio Royalties\Desktop\Project - Apoorva\Python\Scraping\RRC\scraper\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'webapps2.rrc.texas.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [14]:
soup = BeautifulSoup(res_post.content,'html.parser')
# print(soup.prettify())

In [15]:
web_links = soup.select('a')

href = [web_link['href'] for web_link in web_links if 'drillingPermitDetailAction' in web_link['href']]

In [16]:
df = pd.read_html(res_post.text)[9]
df.columns = df.iloc[1,:]
df = df.drop([0,1]).copy()
df = df.reset_index(drop=True)
df['HREF'] = href
df['API NO.'] = int(df['API NO.'].str.split()[0][0])

C:\Users\Apoorva.Saxena\AppData\Local\Temp\ipykernel_39336\3320168950.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(res_post.text)[9]


In [18]:
df

1,API NO.,District,Lease,Well Number,Permitted Operator,County,Status Date,Status Number,Wellbore Profiles,Filing Purpose,Amend,Total Depth,Stacked Lateral Parent Well DP #,Status,HREF
0,38940705,08,RAMONES UNIT 267-266E,18H,CALLON PETROLEUM OPERATING CO(124828),REEVES,Submitted: 08/03/2023 Approved: 08/03/2023,892588,Horizontal,New Drill,N,11000,892586,APPROVED,drillingPermitDetailAction.do?methodToCall=sea...
1,38940705,08,RAMONES 267-266E A1,18H,CALLON PETROLEUM OPERATING CO(124828),REEVES,Submitted: 11/17/2023,892588,Horizontal,New Drill,Y,11000,892586,SWR HOLD,drillingPermitDetailAction.do?methodToCall=sea...


In [20]:
df['Status Date'].apply(
    lambda x: re.findall(
        r'\b(1[0-2]|0[1-9])/(3[01]|[12][0-9]|0[1-9])/([0-9]{4})\b', x) 
    )

0    [(08, 03, 2023), (08, 03, 2023)]
1                    [(11, 17, 2023)]
Name: Status Date, dtype: object